In [1]:
%pip install langchain
%pip install -qU langchain-community faiss-cpu


%pip install torch torchvision
%pip install -U sentence-transformers
%pip install accelerate
%pip install langchain_huggingface


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgr

In [2]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

%pip install -qU langchain_ollama




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -qU langchain_ollama
%pip install -qU pypdf





[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch 

# For those with NVIDIA graphics card
torch.cuda.is_available()

# Apple alternative 
torch.backends.mps.is_available()

if torch.cuda.is_available():
    print("CUDA is available")
    hardware = "cuda"

elif torch.backends.mps.is_available():
    print("MPS is available")
    hardware = "mps"

print(f"Using {hardware} for hardware acceleration")


MPS is available
Using mps for hardware acceleration


In [5]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="qwen2.5:1.5b")

response = llm.invoke("What is the capital of Vietnam?")

print(response)

The capital of Vietnam is Hanoi.


In [6]:
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# files = ["berkeley_opt.pdf", "uscis_opt.pdf"]
loader = PyPDFDirectoryLoader("../examples")
pages = []
async for page in loader.alazy_load():
    pages.append(page)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, # 1000 characters per chunk
    chunk_overlap=200 # 200 characters overlap between chunks
)

chunks = []

for page in pages:
    chunk_list = text_splitter.split_text(page.page_content)
    for chunk in chunk_list:
        chunks.append(chunk)
print("Number of chunks: ", len(chunks))

print(type(chunks[1]))


Number of chunks:  125
<class 'str'>


## Create Vector Database 
### Cache Setup

In [7]:
import os 
import hashlib 
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.vectorstores import FAISS 


def compute_document_hash(text_chunks: list, embedding_model: str) -> str:
    """
    Compute a hash of the document content using the model name.
    Args: 
        text_chunks: list of text chunks
        embedding_model: name of the embedding model
    Returns:
        hash of the document content
    """

    content = "".join(text_chunks) + embedding_model 
    return hashlib.sha256(content.encode()).hexdigest()[:16]


def create_or_load_vector_store(chunks: list, embeddings: HuggingFaceEmbeddings, cache_dir: str) -> FAISS:
    """ Retrieves a vector store from a list of text chunks using the given embeddings. 
    
    Args: 
        chunks: list of text chunks
        embeddings: embeddings model
        cache_dir: directory to save the vector store
    Returns:
        FAISS object
    """

    os.makedirs(cache_dir, exist_ok=True)


    embedding_model_name = embeddings.model_name
    current_hash = compute_document_hash(chunks, embedding_model_name)

    # Check if cached index exists and contains a valid path 
    hash_file = os.path.join(cache_dir, "content_hash.txt")
    index_file = os.path.join(cache_dir, "index.faiss")

    if os.path.exists(index_file) and os.path.exists(hash_file):
        with open(hash_file, "r") as f:
            cached_hash = f.read().strip()

            if current_hash == cached_hash:
                print("Loading cached FAISS index for document retrieval")
                return FAISS.load_local(
                    folder_path= cache_dir,
                    embeddings=embeddings,
                    allow_dangerous_deserialization=True
                )
            
            else:
                print("Cache invalidated: Documents and embeddings have changed.")
                print("Rebuilding vector store for updated documents...")

                # Delete existing index and hash file 
                for file in os.listdir(cache_dir):
                    os.remove(os.path.join(cache_dir, file))
    else:
        print("No valid cache found. Creating new vector store...")

    
    # Save the new index and hash file 
    print(f"Creating a new vector store with {len(chunks)} document chunks...")
    vector_store = FAISS.from_texts(chunks, embedding = embeddings)

    # Save the new index and hash
    vector_store.save_local(cache_dir)
    with open(hash_file, "w") as f:
        f.write(current_hash)
    
    print("Vector store created and cached successfully!")

    return vector_store


### Retriever Setup

In [8]:
embeddings = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs = {"device": hardware}
)

vector_store = create_or_load_vector_store(chunks, embeddings, "../vector_store")

query = "What is OPT and if when should an international student apply for it?"
results = vector_store.search(query, k = 2, search_type = "similarity")
print(results)

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":2}
)

/var/folders/3n/nhd27m7j6plbm19rqj76xr_h0000gn/T/ipykernel_91151/1681077127.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/Users/kientran/Apps/OPT-RAG/rag-opt/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cache invalidated: Documents and embeddings have changed.
Rebuilding vector store for updated documents...
Creating a new vector store with 125 document chunks...
Vector store created and cached successfully!
[Document(id='1d9f3923-74db-464c-8073-16f94cf575df', metadata={}, page_content='Home>Working in the United States>Students and Exchange Visitors>Optional Practical Training (OPT)\nfor F-1 Students\nOptional Practical Training (OPT) for F-1\nStudents\n\uf05aALERT: Please remember that photos submitted to USCIS must be unmounted and unretouched.\nUnretouched means the photos must not be edited or digitally enhanced.\xa0The submission of any\nmounted or retouched images will delay the processing of your application and may prompt USCIS\nto require you to visit an Application Support Center to verify your identity.\nOptional practical training (OPT) is temporary employment that is directly related to an F-1 studentʼs major\narea of study. Eligible students can apply to receive up to 1

### Download Qwen2.5b Model

In [10]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# model_id = "Qwen/Qwen2.5-1.5B-Instruct"
# local_model_path = "../models/qwen2.5-1.5b"

# print(f"Downloading {model_id} to {local_model_path}...")
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.save_pretrained(local_model_path)

# print(f"Downloading model {model_id}...")
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype="auto",
# )
# model.save_pretrained(local_model_path)

# print(f"Model {model_id} downloaded and saved to {local_model_path}")





## RAG Pipeline With OLLAMA

In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline

def load_quantized_model(model_path="./models/qwen2.5-1.5b-instruct", hardware="mps"):
    """Load a quantized model optimized for inference performance."""
    
    # Determine hardware and set up appropriate quantization
    if hardware == "cuda":  # NVIDIA GPU
        # 4-bit quantization for NVIDIA GPUs
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",  # Normal Float 4 - better for language models
            bnb_4bit_use_double_quant=True  # Further compression
        )
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map="auto",
            quantization_config=quantization_config
        )
        
        # Compile model for faster execution on NVIDIA GPUs
        if torch.__version__ >= "2.0.0":
            model = torch.compile(model)
            
    elif hardware == "mps":  # Apple Silicon
        # Apple Silicon doesn't support 4-bit quantization yet
        # but we can optimize loading
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map="auto",
            torch_dtype=torch.float16  # Use half precision
        )
        
        
    else:  # CPU
        # 8-bit is more stable for CPU inference
        quantization_config = BitsAndBytesConfig(
            load_in_8bit=True
        )
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map="auto",
            quantization_config=quantization_config
        )
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    
    # Create optimized pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        repetition_penalty=1.1,
        temperature=0.1,  # Low temperature for factual responses about visa information
        do_sample=True
    )
    
    # Wrap in LangChain
    return HuggingFacePipeline(pipeline=pipe)

## Prompt Engineer

In [12]:
import logging

logging.basicConfig(level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
                    )
logger = logging.getLogger("opt_rag")

from langchain.prompts import ChatPromptTemplate

visa_assistant_prompt = ChatPromptTemplate.from_template("""
You are OPT-RAG, an expert assistant specializing in international student visa regulations and processes in the United States.

## ROLE AND GUIDELINES
- Provide accurate information based ONLY on official USCIS documents, university policies, and government regulations in the context provided
- Focus specifically on visa-related issues: OPT applications, CPT authorization, study/work permits, and visa status questions
- NEVER fabricate information or provide speculative advice on visa matters
- If information is not available in the context, clearly state this limitation
- Always indicate the source of information in your responses
- Avoid legal advice; clarify when questions require consultation with immigration attorneys

## CONTEXT
{context}

## USER INFORMATION
Student status: International student in the United States
Primary concern: Visa and immigration matters

## RESPONSE FORMAT
- Begin with a direct answer to the question
- Provide specific, relevant details from official sources
- Include citation to specific documents/policies when available
- Highlight important deadlines or requirements
- If applicable, mention next steps the student should take
- End with a disclaimer that this information is not legal advice

## QUESTION
{input}
""")

In [13]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

def create_rag_chain(llm, vector_store):
    """Create a RAG chain for the international student visa assistant."""
    
    # Create retriever from vector store
    retriever = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}  # Retrieve top 5 most relevant documents
    )
    
    # Create combine documents chain with our enhanced prompt
    combine_docs_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=visa_assistant_prompt
    )
    
    # Create and return the retrieval chain
    return create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=combine_docs_chain
    )



In [4]:
def create_streaming_rag_chain(llm, vector_store, verbose=True):
    """Create a RAG chain with streaming and debug capabilities.
    
    Args:
        llm: Language model for generation
        vector_store: Vector store for retrieval
        verbose: Whether to print debug information
        
    Returns:
        A RAG chain with streaming and debugging
    """

    from langchain.chains import create_retrieval_chain
    from langchain.chains.combine_documents import create_stuff_documents_chain
    from langchain_core.prompts import ChatPromptTemplate
    from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
    from langchain.callbacks.manager import CallbackManager
    
    # Set up callback manager for streaming and logging
    callbacks = []
    if verbose:
        callbacks.append(StreamingStdOutCallbackHandler())
    
    callback_manager = CallbackManager(callbacks)

    # Create prompt with debugging flags
    
    # Create retriever with debug output
    retriever = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    
    # Create chain with callbacks for streaming
    combine_docs_chain = create_stuff_documents_chain(
        llm=llm.with_config({"callbacks": callbacks}),  # Add streaming to LLM
        prompt=visa_assistant_prompt
    )

    # Create and return retrieval chain
    return create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=combine_docs_chain
    )

In [20]:
import time

user_query = "What is the latest you can file your OPT application?"

rag_chain = create_rag_chain(llm, vector_store)

start_time = time.time()

response = rag_chain.invoke({"input": user_query})

end_time = time.time()

print(f"Time taken: {end_time - start_time} seconds")

print(response["answer"])








2025-04-26 17:05:57,476 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Time taken: 5.237255811691284 seconds
The most recent date on which USCIS has specified as being acceptable for filing an OPT application is March 15th, 2023. This is based on guidance provided by USCIS. As of that date, applications must be received through the mail or electronically.

### Important Notes:
- Ensure all fees are paid.
- Check your immigration status and ensure you meet any residency requirements within the US.
- Make sure to submit all required documents in a timely manner for the application to be processed correctly.

If you need further assistance with filing, please contact the International Student Office at [email].


In [21]:
print(type(response))
print(response.keys())

with open("output.txt", "w") as file:
    file.write(response['answer'])



<class 'dict'>
dict_keys(['input', 'context', 'answer'])
